In [1]:
import pandas as pd
import json
import numpy as np
import difflib as kwrd
pd.options.display.max_rows = 500

In [2]:
def similar(org,name):
    org = [y.replace('-',' ').replace('(',' ').replace(')',' ') for y in list(org['name'])]
    name = name.replace('-',' ').replace('(',' ').replace(')',' ')
    match = kwrd.get_close_matches(name,org,n=3, cutoff=0.5) 
    if len(match) > 0:
        return match[0]
    return None

In [3]:
with open ('../backend/dev/resources/files/organisations.json', 'r') as orgs:
    old = json.loads(orgs.read())

In [4]:
old = pd.DataFrame(old)
old['abbr'] = old['name'].apply(lambda x: '' if len(x.split('(')) == 1 else x.split('(')[1].replace(')',''))
old['lname'] = old['name'].apply(lambda x: x.split('(')[0].lower())

In [5]:
new = pd.read_csv('./new_organisation.csv')
new['abbr'] = new['name'].apply(lambda x: '' if len(x.split('(')) == 1 else x.split('(')[1].replace(')',''))
new['lname'] = new['name'].apply(lambda x: x.split('(')[0].lower())

In [6]:
old['match'] = old['name'].apply(lambda x: similar(new, x))

In [7]:
##old

In [8]:
merged = pd.merge(new, old, on="lname", how="outer", validate="one_to_one").replace([np.nan], [None])
merged['name'] = merged.apply(lambda x: x['name_x'] if x['name_x'] else x['name_y'], axis=1)
merged = merged[['name','country','country_group','type','url']]
merged['url'] = merged['url'].apply(lambda x: [b.strip() for b in x.split('\n') if b.strip() != ''][0] if x else None)
merged = merged[['name','country','country_group','type','url']].sort_values('name')

In [9]:
merged.to_csv('./result/organisation_merged.csv', index=False)

In [10]:
unlisted = merged[merged['type'].isna()]
unlisted.to_csv('./result/organisation_unlisted.csv')

In [11]:
merged.to_json('../backend/dev/resources/files/organisations_new.json', orient='records')

In [12]:
list(merged['country_group'].unique())

[None, 'AF', 'LAC', 'EU', 'AP', 'GLOBAL', 'WA']